In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyfbs

### Define parameters

In [ ]:
eosDD2 = pyfbs.PyEoStable("/media/data/Documents/PhD/B09/B09/EOS_tables/eos_HS_DD2_with_electrons.beta")
eosCausal = pyfbs.PyCausalEoS(1e-10)

In [ ]:
mu = 1.
lam = 0.

### Test single star integration

In [ ]:
myFBS = pyfbs.PyFermionBosonStar.FromParameters(eosDD2, mu)
myFBS.set_initial_conditions(0.0005, 1e-5)
myFBS.bisection(1., 10., max_step=200)
myFBS.evaluate_model()
myFBS.get()

In [ ]:
ax = plt.gca()
myFBS.plot(ax)
ax.set_xscale('log'); ax.set_yscale('log')
ax.set_xlim(left=1e-3)
ax.legend(); ax.grid()

### Compare to Fig 2. of https://arxiv.org/pdf/1606.03035.pdf

In [ ]:
mu = 1.
lam = 0.
rho_c = np.array([0.])
phi_c = np.geomspace(5e-4, 1e-1, 50)
lam = 0.
res = pyfbs.PyMRcurve.from_rhophi_curve(mu, lam, eosDD2, rho_c, phi_c, "")
res = pyfbs.PyMRcurve.calc_TLN_curve(res)

In [ ]:
M = np.array([r.get()['M_T'] for r in res])
Lam = np.array([r.get()['lambda_tidal'] for r in res])

In [ ]:
plt.plot(phi_c, Lam*M**5)
plt.grid()
plt.ylabel("$\lambda M^5$"); plt.ylim(bottom=0, top=1200)
plt.xlabel("$\phi_c$"); plt.xlim(left=0., right=0.05)

### Compare to DD2 k2

In [ ]:
rho_c = np.geomspace(1e-4, 1e-2, 100)
phi_c = np.array([0.])

res = pyfbs.PyMRcurve.from_rhophi_curve(mu, lam, eosDD2, rho_c, phi_c, "")
res = pyfbs.PyMRcurve.calc_TLN_curve(res)

In [ ]:
M = np.array([r.get()['M_T'] for r in res])
R = np.array([r.get()['R_F_0'] for r in res])
k2 = np.array([r.get()['k2'] for r in res])
C = M/R / 1.477

In [ ]:
plt.plot(C, k2)
plt.grid()
plt.xlabel("C"); #plt.ylim(bottom=0, top=1200)
plt.ylabel("$k_2$"); #plt.xlim(left=0., right=0.05)